# Weather Classification using ResNet

## 1. Import Packages

In [1]:
import torch
import torch.nn as nn
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

## 2. Utils

In [2]:
# Set random seed

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed = 59
set_seed(seed)


## 3. Data Preprocessing

### Loading data

In [7]:
# Root directory of dataset
root_dir = '../data/weather-dataset/dataset'
img_paths = []
labels = []
classes = {
    label_idx: class_name for label_idx, class_name in enumerate(
            sorted(os.listdir(root_dir))
    )
}

for label_idx, class_name in classes.items():
    class_dir = os.path.join(root_dir, class_name)
    for img_filename in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_filename)
        img_paths.append(img_path)
        labels.append(label_idx)

### Train - Test Split

In [10]:
val_size = 0.2
test_size = 0.125
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    img_paths, labels,
    test_size=val_size,
    random_state = seed,
    shuffle=is_shuffle
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size=test_size,
    random_state=seed,
    shuffle=is_shuffle
)

### Implement Pytorch datasets

In [11]:
class WeatherDataset(Dataset):
    def __init__(
        self,
        X, y,
        transform=None
    ):
        self.transform = transform
        self.img_paths = X
        self.labels = y
    
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = Image.open(img_path).convert("RGB")
        
        if self.transform:
            img = self.transform(img)
        
        return img, self.labels[index]    

### Implement Image Preprocessing Function

In [13]:
def transform(img, img_size=(224, 224)):
    img = img.resize(img_size)
    img = np.array(img)[..., :3]
    img = torch.tensor(img).permute(2, 0, 1).float()
    normalized_img = img / 255.0
    
    return normalized_img

### Define Dataset object for train - test - val datasets

In [14]:
train_dataset = WeatherDataset(
    X_train, y_train,
    transform=transform
)

val_dataset = WeatherDataset(
    X_val, y_val,
    transform=transform
)

test_dataset = WeatherDataset(
    X_test, y_test,
    transform=transform
)

### Define DataLoader

In [15]:
train_batch_size = 512
test_batch_size = 8

train_loader = DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=test_batch_size,
    shuffle=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=test_batch_size,
    shuffle=False 
)

## 4. Model Construction (ResNet)

In [16]:
class ResidualBlock(nn.Module):
    """
    Residual Block implementation for ResNet architecture.

    This block consists of two convolutional layers with Batch Normalization and ReLU activations,
    along with an optional downsample layer to match dimensions between input and output.

    Attributes:
        conv1 (nn.Conv2d): First convolutional layer with kernel size 3.
        batch_norm1 (nn.BatchNorm2d): Batch normalization after the first convolution.
        conv2 (nn.Conv2d): Second convolutional layer with kernel size 3.
        batch_norm2 (nn.BatchNorm2d): Batch normalization after the second convolution.
        downsample (nn.Sequential): Optional downsampling layer to adjust input dimensions if necessary.
        relu (nn.ReLU): ReLU activation function.

    Args:
        in_channels (int): Number of input channels.
        out_channels (int): Number of output channels.
        stride (int, optional): Stride for the first convolutional layer. Default is 1.
    """
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        # First convolutional layer: reduces dimensions based on stride and extracts features.
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)

        # Second convolutional layer: keeps dimensions consistent and refines features.
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        # Initialize downsample layer as an empty Sequential module.
        self.downsample = nn.Sequential()

        # Downsample the input if stride > 1 or the number of channels changes.
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),  # 1x1 convolution for channel matching.
                nn.BatchNorm2d(out_channels)  # Normalize the downsampled features.
            )

        # ReLU activation function to introduce non-linearity.
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Forward pass for the Residual Block.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_channels, height, width).

        Returns:
            torch.Tensor: Output tensor of shape (batch_size, out_channels, new_height, new_width).
        """
        # Save the input tensor as the shortcut for residual connection.
        shortcut = x.clone()

        # First convolution, normalization, and activation.
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)

        # Second convolution and normalization.
        x = self.conv2(x)
        x = self.batch_norm2(x)

        # Add the shortcut connection (with downsampling if required).
        x += self.downsample(shortcut)

        # Apply final ReLU activation.
        x = self.relu(x)

        return x


In [17]:
class ResNet(nn.Module):
    def __init__(self, residual_block, n_blocks_lst, n_classes):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=3)
        self.conv2 = self.create_layer(residual_block, 64, 64, n_blocks_lst[0], 1)
        self.conv3 = self.create_layer(residual_block, 64, 128, n_blocks_lst[1], 2)
        self.conv4 = self.create_layer(residual_block, 128, 256, n_blocks_lst[2], 2)
        self.conv5 = self.create_layer(residual_block, 256, 512, n_blocks_lst[3], 2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512, n_classes)
    
    def create_layer(self, residual_block, in_channels, out_channels, n_blocks, stride):
        blocks = []
        first_block = residual_block(in_channels, out_channels, stride)
        blocks.append(first_block)
        
        for idx in range (1, n_blocks):
            block = residual_block(out_channels, out_channels, stride)
            blocks.append(block)
        
        block_sequential = nn.Sequential(*blocks)
        
        return block_sequential
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.maxpool(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        
        return x

In [18]:
n_classes = len(list(classes.keys()))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = ResNet(
    residual_block=ResidualBlock,
    n_blocks_lst=[2, 2, 2, 2],
    n_classes=n_classes
).to(device)

In [19]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    losses = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    loss = sum(losses) / len(losses)
    acc = correct / total
    
    return loss, acc